## Testdaten

Mit diesem Skript werden die Testdaten zum Erzeugen des PUFs generiert. Durch das Skript werden die Dateien `fdz_generated_data_dm12.db` und die `.csv`-Dateien im Verzeichnis `test_data/csv/` erzeugt und falls bereits vorhanden überschrieben.

### Fehler in den erzeugten Daten
- VSID ist immer == PSID (eigentlich anderes Format und eine PSID kann mehere VSID haben)
- KV_NR_Kennzeichen ist konstant 1
- SA451 ANZAHLEINHEITENFAKTOR ist konstant 100
- SA651 und SA551 Diagnose sind == ICD_CODE, keine Punktnotation und keine Sonderzeichen
- SA651 und SA551 ICD_Zusatz ist immer leer
- Zu kleine Spanne bei SA751
- Keine echten PLZ
- Jede Variable für sich generiert, keine Abhängigkeiten, keine sinnvollen zusammenhänge


In [28]:
n = 1000

Tabellenfaktoren:
- SA151, SA152,SA153, SA751, SA951, SA999, SA131: 1
- SA451: 10
- SA551: 2
- SA651: 25

In [29]:
import pandas as pd
import random
import numpy as np
import string
import sqlalchemy
import os
from sqlalchemy import Table, MetaData, text
from sqlalchemy.schema import DropTable
import simple_icd_10 as icd

In [30]:
if not os.path.exists("test_data"):
    os.mkdir("test_data")
if not os.path.exists("test_data/csv"):
    os.mkdir("test_data/csv")

In [31]:
engine = sqlalchemy.create_engine("sqlite+pysqlite:///test_data/fdz_generated_data_dm12.db")
conn = engine.connect()

In [32]:
insp = sqlalchemy.inspect(engine)
tables = insp.get_table_names()
for table in tables:
    conn.execute(DropTable(Table(table, MetaData())))

with open("create_tables.sql", "r") as f_tables:
    sql_create_tables = f_tables.read().format(prefix="VBJ", receiving_year=2016,
                                               clearing_year=2016 - 1,
                                               schema="puf")
for statement in sql_create_tables.split(";"):
    conn.execute(text(statement))
conn.commit()

# Pseudonyme

In [33]:
POSSIBLE_CHARACTERS = string.ascii_uppercase + string.ascii_lowercase + string.digits

def generate_pseudonym(length=19):
    return ''.join(random.choices(POSSIBLE_CHARACTERS, k=length))

def generate_pseudonym_array(n_out, length = 19):
    return np.array([generate_pseudonym(length=length) for _ in range(n_out)])




In [34]:
psid_array = generate_pseudonym_array(n_out=n)

In [35]:
def generate_sa151(n_out, psid_array, year = 2016):

    return pd.DataFrame(    {
        "SA151_SATZART": "SA151",
    "SA151_BERICHTSJAHR": year,
    "SA151_AUSGLEICHSJAHR": year,
    "SA151_VSID": psid_array, # Müsste anderes Format sein
    "SA151_PSID": psid_array,
    "SA151_KV_NR_KENNZEICHEN": 1, # Kann in der realität auch 0 sein
    "SA151_GEBURTSJAHR": np.array([np.random.choice(range(1930, year)) for _ in range(n_out)]),
    "SA151_GESCHLECHT": np.array([np.random.choice([1, 2, 3, 4], p=[0.49, 0.49, 0.01, 0.01]) for _ in range(n_out)]),
    "SA151_VERSICHERTENTAGE": np.array([np.random.choice([365, np.random.choice(range(1, 364))], p = [0.95, 0.05]) for _ in range(n_out)]),
    "SA151_VERSTORBEN": np.array([np.random.choice([0, 1], p=[0.95, 0.05]) for _ in range(n_out)]),
    "SA151_VERSICHERTENTAGEKG": np.array([np.random.choice([0, np.random.choice(range(1, 365))], p = [0.9, 0.1]) for _ in range(n_out)])
    })

In [36]:
sa151 = generate_sa151(n_out = n, psid_array=psid_array)

sa151.to_csv("test_data/csv/sa151.csv", index=False)
sa151.to_sql("VBJ2016SA151", con = conn, if_exists='append', index=False)


1000

In [37]:
def generate_sa152(n_out, psid_array, year = 2016):
    return pd.DataFrame(
        {
               "SA152_SATZART": "SA152",
    "SA152_AUSGLEICHSJAHR": year,
    "SA152_BERICHTSJAHR": year, 
    "SA152_VSID": psid_array, # Hat eigentlich eine andere Struktur
    "SA152_PSID": psid_array,
    "SA152_KV_NR_KENNZEICHEN": 1, # Kann eigentlich auch 0 sein
    "SA152_ERWERBSMINDERUNGS_VT": np.array([np.random.choice([0, np.random.choice(range(1, 365))], p = [0.9, 0.1]) for _ in range(n_out)]),
    "SA152_VERSICHERTENTAGEAUSLAND": np.array([np.random.choice([0, np.random.choice(range(1, 365))], p = [0.9, 0.1]) for _ in range(n_out)]),
    "SA152_VERSICHERTENTAGE13II": np.array([np.random.choice([0, np.random.choice(range(1, 365))], p = [0.9, 0.1]) for _ in range(n_out)]),
    "SA152_VERSICHERTENTAGE53IV": np.array([np.random.choice([0, np.random.choice(range(1, 365))], p = [0.9, 0.1]) for _ in range(n_out)])
        }
    )

In [38]:
sa152 = generate_sa152(n_out = n, psid_array=psid_array)

sa152.to_csv("test_data/csv/sa152.csv", index=False)
sa152.to_sql("VBJ2016SA152", con = conn, if_exists='append', index=False)

1000

In [39]:
def generate_sa153(n_out, psid_array, year = 2016):
    return pd.DataFrame(
        {
            "SA153_SATZART": "SA153",
    "SA153_AUSGLEICHSJAHR": year,
    "SA153_BERICHTSJAHR": year,
    "SA153_VSID": psid_array,
    "SA153_PSID": psid_array,
    "SA153_KV_NR_KENNZEICHEN": 1,
    "SA153_EXTRABLUTREINIGUNG": np.array([np.random.choice([0, 1], p=[0.95, 0.05]) for _ in range(n_out)])
        }
    )

In [40]:
sa153 = generate_sa153(n_out=n, psid_array=psid_array)

sa153.to_csv("test_data/csv/sa153.csv", index=False)
sa153.to_sql("VBJ2016SA153", con = conn, if_exists='append', index=False)

1000

In [41]:
def generate_sa451(n_out, psid_array, year = 2016):

    month = pd.Series(np.array([np.random.choice(range(1, 12)) for _ in range(n_out)]), dtype="string").str.pad(2, fillchar='0')
    day = pd.Series(np.array([np.random.choice(range(1, 28)) for _ in range(n_out)]), dtype="string").str.pad(2, fillchar='0') # Valides Datum, aber nie 29 bis 31
    ids = np.random.choice(psid_array, size = n_out, replace = True)
    
    return pd.DataFrame(
        {
    "SA451_SATZART": "SA451",
    "SA451_AUSGLEICHSJAHR": year,
    "SA451_BERICHTSJAHR": year,
    "SA451_VSID": ids,
    "SA451_PSID": ids,
    "SA451_VERORDNUNGSDATUM":  str(year) + month + day,
    "SA451_PHARMAZENTRALNUMMER": np.array([''.join(random.choices(string.digits, k=8)) for _ in range(n_out)]),
    "SA451_ANZAHLEINHEITENFAKTOR": 100 #Normalerweise nicht konstant
        }
    )


In [42]:
sa451 = generate_sa451(n_out=n*10, psid_array=psid_array)

sa451.to_csv("test_data/csv/sa451.csv", index=False)
sa451.to_sql("VBJ2016SA451", con = conn, if_exists='append', index=False)

10000

In [43]:
def generate_sa551(n_out, psid_array, year = 2016):

    month = pd.Series(np.array([np.random.choice(range(1, 12)) for _ in range(n_out)]), dtype="string").str.pad(2, fillchar='0')
    ids = np.random.choice(psid_array, size = n_out, replace = True)
    icd_codes = pd.Series(np.array([np.random.choice(icd.get_all_codes(with_dots = False)) for _ in range(n_out)]), dtype="string")
    icd_zusatz = pd.Series([''], dtype="string") # Ist in der realität nicht immer leer
    sa551 = pd.DataFrame(
        {
                "SA551_SATZART": "SA551",
    "SA551_AUSGLEICHSJAHR": year,
    "SA551_BERICHTSJAHR": year,
    "SA551_VSID": ids,
    "SA551_PSID": ids,
    "SA551_ENTLASSUNGSMONAT": str(year) + month,
    "SA551_FALLZAEHLER": 1,
    "SA551_DIAGNOSE": icd_codes, # Hat eigentlich Punkt Notation und Sonderzeichen
    "SA551_ICD_CODE": icd_codes,
    "SA551_ICD_ZUSATZ": icd_zusatz,
    "SA551_LOKALISATION": np.array([np.random.choice([0, 1], p=[0.95, 0.05]) for _ in range(n_out)]),
    "SA551_ARTDIAGNOSE": np.array([np.random.choice([1, 2], p=[0.95, 0.05]) for _ in range(n_out)]),
    "SA551_ARTBEHANDLUNG": np.array([np.random.choice([0, 1, 2, 3]) for _ in range(n_out)])
        }
    )

    sa551["SA551_FALLZAEHLER"] = sa551.groupby("SA551_PSID")["SA551_FALLZAEHLER"].rank(method ="first", ascending = True)
    
    return sa551

In [44]:
sa551 = generate_sa551(n_out=n*2, psid_array=psid_array)

sa551.to_csv("test_data/csv/sa551.csv", index=False)
sa551.to_sql("VBJ2016SA551", con = conn, if_exists='append', index=False)

2000

In [45]:
def generate_sa651(n_out, psid_array, year = 2016):

    ids = np.random.choice(psid_array, size = n_out, replace = True)
    icd_codes = pd.Series(np.array([np.random.choice(icd.get_all_codes(with_dots = False)) for _ in range(n_out)]), dtype="string")
    icd_zusatz = pd.Series([''], dtype="string") # Ist in der realität nicht immer leer
    return pd.DataFrame(
        {
                "SA651_SATZART": "SA651",
    "SA651_AUSGLEICHSJAHR": year,
    "SA651_BERICHTSJAHR": year,
    "SA651_VSID": ids,
    "SA651_PSID": ids,
    "SA651_LEISTUNGSQUARTAL": np.array([np.random.choice(range(1, 4)) for _ in range(n_out)] ),
    "SA651_DIAGNOSE": icd_codes, # Hat eigentlich Punkt Notation und Sonderzeichen
    "SA651_ICD_CODE": icd_codes,
    "SA651_ICD_ZUSATZ": icd_zusatz,
    "SA651_QUALIFIZIERUNG": np.array([np.random.choice(["V", "Z", "A", "G", "0"]) for _ in range(n_out)]),
    "SA651_LOKALISATION": np.array([np.random.choice([0, 1], p=[0.95, 0.05]) for _ in range(n_out)]),
    "SA651_ABRECHNUNGSWEG": np.array([np.random.choice([1, 2, 3]) for _ in range(n_out)])
        }
    )

In [46]:
sa651 = generate_sa651(n_out=n*25, psid_array=psid_array)

sa651.to_csv("test_data/csv/sa651.csv", index=False)
sa651.to_sql("VBJ2016SA651", con = conn, if_exists='append', index=False)

25000

In [47]:
def generate_sa751(n_out, psid_array, year = 2016):
    return pd.DataFrame(
        {
    "SA751_SATZART": "SA751",
    "SA751_AUSGLEICHSJAHR": year,
    "SA751_BERICHTSJAHR": year,
    "SA751_VSID": psid_array,
    "SA751_PSID": psid_array,
    "SA751_AERZTE": np.array([np.random.randint(low = -100000, high = 100000) for _ in range(n_out)]), # viel zu kleine spanne
	"SA751_ZAHNAERZTE": np.array([np.random.randint(low = -100000, high = 100000) for _ in range(n_out)]), # viel zu kleine spanne
	"SA751_APOTHEKEN": np.array([np.random.randint(low = -100000, high = 100000) for _ in range(n_out)]), # viel zu kleine spanne
	"SA751_KRANKENHAEUSER": np.array([np.random.randint(low = -100000, high = 100000) for _ in range(n_out)]), # viel zu kleine spanne
	"SA751_SONSTIGELA": np.array([np.random.randint(low = -100000, high = 100000) for _ in range(n_out)]), # viel zu kleine spanne
	"SA751_SACHKOSTENDIALYSE": np.array([np.random.randint(low = -100000, high = 100000) for _ in range(n_out)]), # viel zu kleine spanne
	"SA751_KRANKENGELD": np.array([np.random.randint(low = -100000, high = 100000) for _ in range(n_out)]) # viel zu kleine spanne
        }
    )

In [48]:
sa751 = generate_sa751(n_out = n, psid_array=psid_array)

sa751.to_csv("test_data/csv/sa751.csv", index=False)
sa751.to_sql("VBJ2016SA751", con = conn, if_exists='append', index=False)

1000

In [49]:
def generate_sa951(n_out, psid_array, year = 2016):
    betriebsnummern = np.array([''.join(random.choices(string.digits, k=8)) for _ in range(50)])
             
    return pd.DataFrame(
        {
    "SA951_SATZART": "SA951",
    "SA951_AUSGLEICHSJAHR": year,
    "SA951_BERICHTSJAHR": year,
    "SA951_VSID": psid_array,
    "SA951_PSID": psid_array,
    "SA951_BETRIEBSNUMMER": np.random.choice(betriebsnummern, size = n_out, replace = True)
        }
    )

In [50]:
sa951 = generate_sa951(n_out = n, psid_array=psid_array)

sa951.to_csv("test_data/csv/sa951.csv", index=False)
sa951.to_sql("VBJ2016SA951", con = conn, if_exists='append', index=False)

1000

In [51]:
def generate_sa131(n_out, psid_array, year = 2016):      
    plzs = np.array([''.join(random.choices(string.digits, k=5)) for _ in range(50)])      
    return pd.DataFrame(
        {
    "SA131_SATZART": "SA951",
    "SA131_AUSGLEICHSJAHR": year,
    "SA131_BERICHTSJAHR": year,
    "SA131_VSID": psid_array,
    "SA131_PSID": psid_array,
    "SA131_PLZ": np.random.choice(plzs, size = n_out, replace = True) # Keine Echten PLZs
        }
    )

In [52]:
sa131 = generate_sa131(n_out=n, psid_array=psid_array)

sa131.to_csv("test_data/csv/sa131.csv", index=False)
sa131.to_sql("V2016SA131", con = conn, if_exists='append', index=False)

1000

In [53]:
conn.close()